# Mesh subregions

Often we need to define subregions in the mesh. For instance, this can be if when we have two or more different materials in our mesh and it can make our life easier later when start defining fields and assigning different parameters to different subregions.

As before, the region of the mesh we are going to use as an example is:

$$\mathbf{p}_{1} = (0, 0, 0)$$
$$\mathbf{p}_{2} = (l_{x}, l_{y}, l_{z})$$

with $l_{x} = 100 \,\text{nm}$, $l_{y} = 50 \,\text{nm}$, and $l_{z} = 20 \,\text{nm}$, and we discretise it with cell $(5\,\text{nm}, 5\,\text{nm}, 5\,\text{nm})$.

Subregions in the mesh are defined by passing `subregions` attribute. It is a Python dictionary, whose keys are the names of subregions, whereas the values are the subregion objects. Names of subregions must be valid Python variable name strings (no spaces, no dashes, etc.) and the subregions are the region objects we introduced already.

As an example, let us say we have two subregions of the same size, stacked on top of each other in the z-direction. More precisely, points `p1` and `p2` of two subregions would be:

**Subregion 1:**

$\mathbf{p}_{1} = (0, 0, 0)$

$\mathbf{p}_{2} = (l_{x}, l_{y}, l_{z}/2)$

**Subregion 2:**

$\mathbf{p}_{1} = (0, 0, l_{z}/2)$

$\mathbf{p}_{2} = (l_{x}, l_{y}, l_{z})$

Let us name our subregions 1 and 2 to be "bottom_subregion" and "top_subregion". Accordingly, subregions dictionary is:

In [1]:
import discretisedfield as df

lx, ly, lz = 100e-9, 50e-9, 20e-9

subregions = {'bottom_region': df.Region(p1=(0, 0, 0), p2=(lx, ly, lz/2)),
              'top_region': df.Region(p1=(0, 0, lz/2), p2=(lx, ly, lz))}

Having the subregions dictionary, we can now pass it to our mesh:

In [2]:
cell = (5e-9, 5e-9, 5e-9)

region = df.Region(p1=(0, 0, 0), p2=(lx, ly, lz))
mesh = df.Mesh(region=region, cell=cell, subregions=subregions)

The mesh with two subregions is now defined. Please note that it is your responsibility that the subregions are well defined and "make sense". If they overlap or do not cover the entire mesh region, no errors or warnings will be raised. This gives a lot of freedom to users how they can define the subregions, but also some responisibilites.

Regarding the mesh operations we can perform, all the ones we showed before are still there, but now we can use subregion-specific ones as well. We can ask for the mesh subregions:

In [3]:
mesh.subregions

{'bottom_region': Region(p1=(0.0, 0.0, 0.0), p2=(1e-07, 5e-08, 1e-08)),
 'top_region': Region(p1=(0.0, 0.0, 1e-08), p2=(1e-07, 5e-08, 2e-08))}

This is simply a dictionary we used at the mesh definition. From the mesh we can extract a subregion mesh, by "indexing" it with the subregion name.

In [4]:
mesh['bottom_region']

Mesh(region=Region(p1=(0.0, 0.0, 0.0), p2=(1e-07, 5e-08, 1e-08)), n=(20, 10, 2), bc='', subregions={})

This returns a mesh defined on a subregion and with the same cell size we used at the mesh definition. So, we expect that one half of the total number of cells belong to each subregion:

In [5]:
len(mesh)  # total number of cells

800

In [6]:
len(mesh['bottom_region'])  # number of cells in the bottom_region

400

In [7]:
len(mesh['bottom_region'])  # number of cells in the top_region

400

The object indexing operator returns is a mesh and we can perform all the typical mesh operations on it. For example, let us say we want to get the centre point of the `bottom_region`. First we need to extract the subregion (`[]`), then access its region (`.region`) and finally ask for its centre point (`.centre`):

In [8]:
mesh['bottom_region'].region.centre

(5e-08, 2.5e-08, 5e-09)

### Mesh subregions visualisation

The mesh is defined. Based on the region dimensions and the number of discretisation cells, we can ask the mesh to give us the size of a single discretisation cell:

In [27]:
mesh.cell

(2e-08, 2.5e-08, 2e-08)

Knowing this value, we could have defined the mesh passing this value using `cell` argument, and we would have got exactly the same mesh.

In [28]:
cell = (20e-9, 25e-9, 20e-9)

mesh = df.Mesh(region=region, cell=cell)

If we now ask our new mesh about the number of discretisation cells:

In [29]:
mesh.n

(5, 2, 1)

There is no difference whatsoever how we are going to define the mesh. However, defining the mesh with `cell` can result in an error, if the region cannot be divided into chunks of that size. For instance:

In [30]:
try:
    mesh = df.Mesh(region=region, cell=(3e-9, 3e-9, 3e-9))
except ValueError:
    print('Exception raised.')

Exception raised.


Let us now have a look at some basic properties we can ask the mesh object for. First of all, region object is a part of the mesh object:

In [31]:
mesh.region

Region(p1=(0.0, 0.0, 0.0), p2=(1e-07, 5e-08, 2e-08))

Therefore, we can perform all the operations on the region we saw previously, but now through the mesh object (`mesh.region`). For instance: 

In [32]:
mesh.region.pmin  # minimum point

(0.0, 0.0, 0.0)

In [33]:
mesh.region.edges  # edge lenghts

(1e-07, 5e-08, 2e-08)

In [34]:
mesh.region.centre  # centre point

(5e-08, 2.5e-08, 1e-08)

By asking the mesh object directly, we can get the number of discretisation cells in all three directions $n = (n_{x}, n_{y}, n_{z})$:

In [35]:
mesh.n

(5, 2, 1)

and the size of a single discretisation cell:

In [36]:
mesh.cell

(2e-08, 2.5e-08, 2e-08)

The total number of discretisation cells is:

In [37]:
len(mesh)

10

This number is simply $n_{x}n_{y}n_{z}$. We can conclude that the entire region is now divided into 100 small cubes (discretisation cells). Each cell in the mesh has its index and its coordinate. We can get indices of all discretisation cells:

In [38]:
mesh.indices

<generator object Mesh.indices at 0x7fe797503120>

This gives us a generator object we can use as an iterable in different Pyhton contexts. For instance, we can give it to the `list`.

In [39]:
list(mesh.indices)

[(0, 0, 0),
 (1, 0, 0),
 (2, 0, 0),
 (3, 0, 0),
 (4, 0, 0),
 (0, 1, 0),
 (1, 1, 0),
 (2, 1, 0),
 (3, 1, 0),
 (4, 1, 0)]

List function now "unpacks" the generator and gives us a list of tuples. Each tuple has three unsigned integers. For instance, we can interpret index `(2, 1, 0)` as an index which belongs to the third cell in the x-direction, second in the y, and the first in the z direction. Please note that indexing in Python starts from 0. Therefore, we say that the "fifth element" has index 4.

Another thing we can associate to every discretisation cell is its coordinate. The coordinate of the cell is the coordinate of its centre point. So, the coordinate of index `(2, 1, 0)` cell is:

In [41]:
index = (2, 1, 0)

mesh.index2point(index)

(5e-08, 3.75e-08, 1e-08)

It is very often the case we need to iterate through all discretisation cells and use their coordinates. For that, we can use the mesh object itself, which is also an iterable:

In [42]:
list(mesh)

[(1e-08, 1.25e-08, 1e-08),
 (3.0000000000000004e-08, 1.25e-08, 1e-08),
 (5e-08, 1.25e-08, 1e-08),
 (7e-08, 1.25e-08, 1e-08),
 (9e-08, 1.25e-08, 1e-08),
 (1e-08, 3.75e-08, 1e-08),
 (3.0000000000000004e-08, 3.75e-08, 1e-08),
 (5e-08, 3.75e-08, 1e-08),
 (7e-08, 3.75e-08, 1e-08),
 (9e-08, 3.75e-08, 1e-08)]

Since mesh object is an iterator itself, we can use it, for example, in for loops:

In [43]:
for point in mesh:
    print(point)

(1e-08, 1.25e-08, 1e-08)
(3.0000000000000004e-08, 1.25e-08, 1e-08)
(5e-08, 1.25e-08, 1e-08)
(7e-08, 1.25e-08, 1e-08)
(9e-08, 1.25e-08, 1e-08)
(1e-08, 3.75e-08, 1e-08)
(3.0000000000000004e-08, 3.75e-08, 1e-08)
(5e-08, 3.75e-08, 1e-08)
(7e-08, 3.75e-08, 1e-08)
(9e-08, 3.75e-08, 1e-08)


A function, which is opposite to `index2point`, is `point2index`. This function takes any point in the region and returns the index of a cell it belongs to:

In [46]:
point = (41.6e-9, 35.2e-9, 4.71e-9)

mesh.point2index(point)

(2, 1, 0)

We can compare meshes using `==` and `!=` relational operators. Let us define two meshes and compare them to the one we have already defined:

In [52]:
mesh_same = df.Mesh(region=region, n=(5, 2, 1))
mesh_different = df.Mesh(region=region, n=(10, 5, 7))

mesh == mesh_same

True

In [50]:
mesh == mesh_different

False

In [53]:
mesh != mesh_different

True

Finally, mesh has its representation string:

In [54]:
mesh

Mesh(region=Region(p1=(0.0, 0.0, 0.0), p2=(1e-07, 5e-08, 2e-08)), n=(5, 2, 1), bc='', subregions={})

In the representation string, we see `p1`, `p2`, and `n` we discussed earlier, but there are also `bc` and `subregions` we did not and we will look at some more advanced mesh properties in the next tutorials.